pca_fasttext_150features

In [1]:
from google.colab import drive
drive.mount('/content/drive') 

Mounted at /content/drive


In [2]:
import warnings
warnings.filterwarnings("ignore") 

In [3]:
import os 
import numpy as np 
import pandas as pd
from sklearn import preprocessing
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, classification_report
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.neighbors import KNeighborsClassifier

In [4]:
import pandas as pd
path = "/content/drive/MyDrive/newpro/threeclass_problem/PCA_Fasttext/data.csv"
df = pd.read_csv(path)
df.head() 

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,Level 5,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,Level 5,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,Level 5,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",Level 5,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,Level 2,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [5]:
df.iloc[:,1].replace(['Level 1', 'Level 2', 'Level 3', 'Level 4', 'Level 5'],
                        [1, 2, 3, 4, 5], inplace=True)

In [6]:
df.head()

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,5,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,5,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,5,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",5,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,2,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [7]:
#define bins
bins = [0,2,4,5]
#name of the group 
group_names = ['Low', 'Moderate', 'High']
df['Level '] = pd.cut(df["Level "],bins,labels = group_names)

In [8]:
df.head(5)

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,High,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,High,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,High,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",High,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,Low,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [9]:
df['Level '] = df['Level '].map({'Low': 1, 'Moderate': 2, 'High': 3}).astype(int)

In [10]:
df.head()

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,3,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,3,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,3,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",3,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,1,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [11]:
import pandas as pd
path = "/content/drive/MyDrive/newpro/threeclass_problem/PCA_Fasttext/fasttext_withstopwordchangelevels_150_features.csv"
x = pd.read_csv(path)
x.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,140,141,142,143,144,145,146,147,148,149
0,0,-7.005115,-3.385458,-2.191794,2.811184,0.256253,-1.158846,6.165057,-0.634567,4.651401,...,-4.322727e-07,-2.687812e-07,9.928342e-07,2.893616e-07,6.141707e-07,4.665733e-08,2.632960e-07,1.102826e-06,-1.779100e-07,-6.677595e-08
1,1,-8.306421,2.676702,-1.245733,1.885322,2.348943,1.929765,9.834216,1.037862,-5.207885,...,2.515014e-07,-4.504684e-07,-9.240621e-08,1.124187e-07,-8.290994e-07,3.187187e-07,-3.382103e-07,8.450565e-07,-5.568892e-07,-1.253165e-07
2,2,6.553153,-2.947551,-8.255116,0.875345,3.352671,0.129313,-3.940301,4.933846,-3.690879,...,-5.317946e-07,-7.862854e-08,-6.381471e-07,1.125670e-07,1.893926e-08,-2.042682e-08,1.418043e-07,-2.329346e-12,-4.311012e-07,1.875010e-07
3,3,12.606214,-4.158964,12.214924,13.384568,-1.207894,-2.059413,-2.372467,6.299860,-3.473671,...,-4.142922e-07,6.567411e-08,-1.165567e-07,3.357173e-07,2.137854e-07,3.195252e-07,3.172300e-07,-1.536424e-07,1.837729e-07,4.794247e-07
4,4,-7.945574,2.975537,-1.333256,-1.101897,0.665095,4.372155,-0.469863,-2.851719,-0.176283,...,1.294733e-07,1.079266e-06,4.178303e-07,-6.430464e-07,1.208701e-07,-7.912780e-07,2.099251e-07,2.133294e-07,1.134899e-06,2.804971e-07


In [12]:
x.drop(x.columns[[0]], axis=1, inplace=True)
x.head()

,0,1,2,3,4,5,6,7,8,9,...,140,141,142,143,144,145,146,147,148,149
0,-7.005115,-3.385458,-2.191794,2.811184,0.256253,-1.158846,6.165057,-0.634567,4.651401,3.778039,...,-4.322727e-07,-2.687812e-07,9.928342e-07,2.893616e-07,6.141707e-07,4.665733e-08,2.632960e-07,1.102826e-06,-1.779100e-07,-6.677595e-08
1,-8.306421,2.676702,-1.245733,1.885322,2.348943,1.929765,9.834216,1.037862,-5.207885,0.576728,...,2.515014e-07,-4.504684e-07,-9.240621e-08,1.124187e-07,-8.290994e-07,3.187187e-07,-3.382103e-07,8.450565e-07,-5.568892e-07,-1.253165e-07
2,6.553153,-2.947551,-8.255116,0.875345,3.352671,0.129313,-3.940301,4.933846,-3.690879,1.130900,...,-5.317946e-07,-7.862854e-08,-6.381471e-07,1.125670e-07,1.893926e-08,-2.042682e-08,1.418043e-07,-2.329346e-12,-4.311012e-07,1.875010e-07
3,12.606214,-4.158964,12.214924,13.384568,-1.207894,-2.059413,-2.372467,6.299860,-3.473671,0.953171,...,-4.142922e-07,6.567411e-08,-1.165567e-07,3.357173e-07,2.137854e-07,3.195252e-07,3.172300e-07,-1.536424e-07,1.837729e-07,4.794247e-07
4,-7.945574,2.975537,-1.333256,-1.101897,0.665095,4.372155,-0.469863,-2.851719,-0.176283,4.778185,...,1.294733e-07,1.079266e-06,4.178303e-07,-6.430464e-07,1.208701e-07,-7.912780e-07,2.099251e-07,2.133294e-07,1.134899e-06,2.804971e-07


In [13]:
y=df['Level '] 

**Train Test Split**

In [14]:
# Train-Test-Split is performed

from sklearn.model_selection import train_test_split

# Splitting the data into train and test
X_train, X_test, y_train, y_test = train_test_split(x, y, train_size=0.7, random_state=42)
X_train.shape, X_test.shape

((1220, 150), (524, 150))

In [15]:
print(y_train.value_counts(normalize=True)*100)
print("\n")
print(y_test.value_counts(normalize=True)*100)

2    45.327869
1    29.590164
3    25.081967
Name: Level , dtype: float64


2    45.229008
1    29.961832
3    24.809160
Name: Level , dtype: float64


**Model Building**

In [ ]:
def perceptron_classifier (X_train, y_train, X_test, y_test):
  p = Perceptron(random_state=150)
  p.fit(X_train, y_train)
  predictions_train = p.predict(X_train)
  predictions_test = p.predict(X_test)
  train_score = accuracy_score(predictions_train, y_train)
  #print("score on train data: ", train_score)
  test_score = f1_score(predictions_test, y_test, average='weighted' )
  print("Perceptron F1 score:", test_score)

In [ ]:
from sklearn.linear_model import Perceptron
perceptron_classifier(X_train, y_train, X_test, y_test) 

Perceptron F1 score: 0.3730034179256995


**Decision Tree**

In [16]:
clf_dt = DecisionTreeClassifier(max_depth=5, class_weight = "balanced")

In [17]:
clf_dt.fit(X_train,y_train)

DecisionTreeClassifier(class_weight='balanced', max_depth=5)

In [18]:
y_pred_train = clf_dt.predict(X_train)
y_pred_test = clf_dt.predict(X_test)

In [19]:
def classifcation_report_train_test(y_train, y_train_pred, y_test, y_test_pred):

    print('''
            =========================================
               CLASSIFICATION REPORT FOR TRAIN DATA
            =========================================
            ''')
    print(classification_report(y_train, y_train_pred, digits=4))

    print('''
            =========================================
               CLASSIFICATION REPORT FOR TEST DATA
            =========================================
            ''')
    print(classification_report(y_test, y_test_pred, digits=4))

In [20]:
scores = pd.DataFrame(columns=['Model', 'Train_Accuracy', 'Train_Recall', 'Train_Precision', 'Train_F1_Score', 
                               'Test_Accuracy', 'Test_Recall', 'Test_Precision', 'Test_F1_Score'])

def get_metrics(train_actual, train_predicted, test_actual, test_predicted, model_description, dataframe):

    train_accuracy  = accuracy_score(train_actual, train_predicted)
    train_recall    = recall_score(train_actual, train_predicted, average="weighted")
    train_precision = precision_score(train_actual, train_predicted, average="weighted")
    train_f1score   = f1_score(train_actual, train_predicted, average="weighted")
    
    test_accuracy   = accuracy_score(test_actual, test_predicted)
    test_recall     = recall_score(test_actual, test_predicted, average="weighted")
    test_precision  = precision_score(test_actual, test_predicted, average="weighted")
    test_f1score    = f1_score(test_actual, test_predicted, average="weighted")

    dataframe       = dataframe.append(pd.Series([model_description, 
                                                  train_accuracy, train_recall, train_precision, train_f1score,
                                                  test_accuracy, test_recall, test_precision, test_f1score],
                                                 index=scores.columns ), 
                                       ignore_index=True)

    return(dataframe)

In [21]:
scores = get_metrics(y_train, y_pred_train, y_test, y_pred_test, "DecisionTrees", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.409016,0.409016,0.609304,0.343426,0.278626,0.278626,0.283328,0.222622


**Random Forest**

In [22]:
clf_rf = RandomForestClassifier(class_weight = "balanced")

In [23]:
clf_rf.fit(X=X_train, y=y_train)

RandomForestClassifier(class_weight='balanced')

In [24]:
y_pred_train1 = clf_rf.predict(X_train)
y_pred_test1 = clf_rf.predict(X_test)

In [25]:
classifcation_report_train_test(y_train, y_pred_train1, y_test, y_pred_test1)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.9972    1.0000    0.9986       361
           2     1.0000    0.9964    0.9982       553
           3     0.9967    1.0000    0.9984       306

    accuracy                         0.9984      1220
   macro avg     0.9980    0.9988    0.9984      1220
weighted avg     0.9984    0.9984    0.9984      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.2222    0.0637    0.0990       157
           2     0.4292    0.8312    0.5661       237
           3     0.2500    0.0385    0.0667       130

    accuracy                         0.4046       524
   macro avg     0.3005    0.3111    0.2439       524
weighted avg     0.3227    0.4046    0.3022       524



In [26]:
scores = get_metrics(y_train, y_pred_train1, y_test, y_pred_test1, "RandomForest", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.409016,0.409016,0.609304,0.343426,0.278626,0.278626,0.283328,0.222622
1,RandomForest,0.998361,0.998361,0.998366,0.998360,0.404580,0.404580,0.322725,0.302242


**AdaBoost**

In [ ]:
clf_adaboost =  AdaBoostClassifier()

In [ ]:
clf_adaboost.fit(X_train, y_train)

AdaBoostClassifier()

In [ ]:
y_pred_train2 = clf_adaboost.predict(X_train)
y_pred_test2 = clf_adaboost.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train2, y_test, y_pred_test2)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.5390    0.4017    0.4603       361
           2     0.5485    0.7667    0.6395       553
           3     0.5730    0.3333    0.4215       306

    accuracy                         0.5500      1220
   macro avg     0.5535    0.5006    0.5071      1220
weighted avg     0.5519    0.5500    0.5318      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.2857    0.2675    0.2763       157
           2     0.4253    0.5527    0.4807       237
           3     0.2174    0.1154    0.1508       130

    accuracy                         0.3588       524
   macro avg     0.3095    0.3119    0.3026       524
weighted avg     0.3319    0.3588    0.3376       524



In [ ]:
scores = get_metrics(y_train, y_pred_train2, y_test, y_pred_test2, "Adaboost", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.518852,0.518852,0.566655,0.465471,0.402672,0.402672,0.335762,0.336720
1,RandomForest,0.998361,0.998361,0.998367,0.998361,0.408397,0.408397,0.330830,0.328297
2,Adaboost,0.550000,0.550000,0.551858,0.531806,0.358779,0.358779,0.331908,0.337621


**Gradient Boosting Classifier**

In [ ]:
clf_gbm = GradientBoostingClassifier()

In [ ]:
clf_gbm.fit(X=X_train, y=y_train)

GradientBoostingClassifier()

In [ ]:
y_pred_train3 = clf_gbm.predict(X_train)
y_pred_test3 = clf_gbm.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train3, y_test, y_pred_test3)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.9741    0.8338    0.8985       361
           2     0.8109    0.9928    0.8927       553
           3     1.0000    0.7647    0.8667       306

    accuracy                         0.8885      1220
   macro avg     0.9283    0.8638    0.8860      1220
weighted avg     0.9066    0.8885    0.8879      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.2396    0.1465    0.1818       157
           2     0.4529    0.7300    0.5590       237
           3     0.3696    0.1308    0.1932       130

    accuracy                         0.4065       524
   macro avg     0.3540    0.3357    0.3113       524
weighted avg     0.3683    0.4065    0.3552       524



In [ ]:
scores = get_metrics(y_train, y_pred_train3, y_test, y_pred_test3, "GBM", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.518852,0.518852,0.566655,0.465471,0.402672,0.402672,0.335762,0.336720
1,RandomForest,0.998361,0.998361,0.998367,0.998361,0.408397,0.408397,0.330830,0.328297
2,Adaboost,0.550000,0.550000,0.551858,0.531806,0.358779,0.358779,0.331908,0.337621
3,GBM,0.888525,0.888525,0.906638,0.887881,0.406489,0.406489,0.368303,0.355218


**SVM Model**

In [27]:
linear_svm = SVC(kernel='linear', C=1, class_weight = "balanced")

In [28]:
linear_svm.fit(X=X_train, y= y_train)

SVC(C=1, class_weight='balanced', kernel='linear')

In [29]:
y_pred_train4 = linear_svm.predict(X_train)
y_pred_test4 = linear_svm.predict(X_test)

In [30]:
classifcation_report_train_test(y_train, y_pred_train4, y_test, y_pred_test4)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.3914    0.4543    0.4205       361
           2     0.5459    0.3978    0.4603       553
           3     0.3392    0.4412    0.3835       306

    accuracy                         0.4254      1220
   macro avg     0.4255    0.4311    0.4214      1220
weighted avg     0.4483    0.4254    0.4292      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.3010    0.3949    0.3416       157
           2     0.4248    0.2743    0.3333       237
           3     0.2364    0.3000    0.2644       130

    accuracy                         0.3168       524
   macro avg     0.3207    0.3231    0.3131       524
weighted avg     0.3410    0.3168    0.3187       524



In [31]:
scores = get_metrics(y_train, y_pred_train4, y_test, y_pred_test4, "SVM", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.409016,0.409016,0.609304,0.343426,0.278626,0.278626,0.283328,0.222622
1,RandomForest,0.998361,0.998361,0.998366,0.998360,0.404580,0.404580,0.322725,0.302242
2,SVM,0.425410,0.425410,0.448343,0.429247,0.316794,0.316794,0.340966,0.318709


**KNN**

In [ ]:
knn = KNeighborsClassifier(n_neighbors=7)

In [ ]:
knn.fit(X=X_train, y=y_train)

KNeighborsClassifier(n_neighbors=7)

In [ ]:
y_pred_train5 = knn.predict(X_train)
y_pred_test5 = knn.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train5, y_test, y_pred_test5)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.5041    0.5125    0.5082       361
           2     0.5806    0.7161    0.6413       553
           3     0.4503    0.2516    0.3229       306

    accuracy                         0.5393      1220
   macro avg     0.5117    0.4934    0.4908      1220
weighted avg     0.5253    0.5393    0.5221      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.2857    0.2675    0.2763       157
           2     0.4219    0.5359    0.4721       237
           3     0.2105    0.1231    0.1553       130

    accuracy                         0.3531       524
   macro avg     0.3061    0.3088    0.3013       524
weighted avg     0.3287    0.3531    0.3349       524



In [ ]:
scores = get_metrics(y_train, y_pred_train5, y_test, y_pred_test5, "KNN", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.518852,0.518852,0.566655,0.465471,0.402672,0.402672,0.335762,0.336720
1,RandomForest,0.998361,0.998361,0.998367,0.998361,0.408397,0.408397,0.330830,0.328297
2,Adaboost,0.550000,0.550000,0.551858,0.531806,0.358779,0.358779,0.331908,0.337621
3,GBM,0.888525,0.888525,0.906638,0.887881,0.406489,0.406489,0.368303,0.355218
4,SVM,0.453279,0.453279,0.330837,0.284192,0.452290,0.452290,0.204566,0.281716
5,KNN,0.539344,0.539344,0.525296,0.522053,0.353053,0.353053,0.328668,0.334862


**GNB**

In [ ]:
from sklearn.naive_bayes import GaussianNB
nb = GaussianNB()

In [ ]:
nb.fit(X=X_train, y=y_train)

GaussianNB()

In [ ]:
y_pred_train6 = nb.predict(X_train)
y_pred_test6 = nb.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train6, y_test, y_pred_test6)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.3513    0.6510    0.4563       361
           2     0.5293    0.4087    0.4612       553
           3     0.3952    0.1601    0.2279       306

    accuracy                         0.4180      1220
   macro avg     0.4252    0.4066    0.3818      1220
weighted avg     0.4430    0.4180    0.4012      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.3003    0.5605    0.3911       157
           2     0.4341    0.3333    0.3771       237
           3     0.2245    0.0846    0.1229       130

    accuracy                         0.3397       524
   macro avg     0.3196    0.3262    0.2970       524
weighted avg     0.3420    0.3397    0.3182       524



In [ ]:
scores = get_metrics(y_train, y_pred_train6, y_test, y_pred_test6, "GNB", scores)
scores

,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.518852,0.518852,0.566655,0.465471,0.402672,0.402672,0.335762,0.336720
1,RandomForest,0.998361,0.998361,0.998367,0.998361,0.408397,0.408397,0.330830,0.328297
2,Adaboost,0.550000,0.550000,0.551858,0.531806,0.358779,0.358779,0.331908,0.337621
3,GBM,0.888525,0.888525,0.906638,0.887881,0.406489,0.406489,0.368303,0.355218
4,SVM,0.453279,0.453279,0.330837,0.284192,0.452290,0.452290,0.204566,0.281716
5,KNN,0.539344,0.539344,0.525296,0.522053,0.353053,0.353053,0.328668,0.334862
6,GNB,0.418033,0.418033,0.442964,0.401250,0.339695,0.339695,0.342006,0.318229
